In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

from nltk.tag.stanford import StanfordNERTagger
nlp_small = spacy.load('en_core_web_sm',disable=["ner"])
from spacy.symbols import VERB

In [48]:
%run getMostSimilarTag.ipynb
%run javaCommunicator.ipynb

No embedding in google model
No embedding in wiki model


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)


In [5]:
def show_train_res(corpus, model='english.muc.7class.distsim.crf.ser'):
    jar = 'stnf/stanford-ner.jar'

    ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

    words = word_tokenize(corpus)
    return ner_tagger.tag(words)

In [29]:
model = "english.muc.7class.distsim.crf.ser"#"dummy-ner-model.ser.gz"
loc_dat = input("Where and when are u planing to go?\n")
activity = input("And what are going to do there?\n") #"I'm going to meet friends and have a tasty burger with them"
ents = show_train_res(loc_dat, model)
ents_act = show_train_res(activity, model)

loc = " ".join([ent[0] for ent in ents if ent[1]=="LOCATION"])
date = " ".join([ent[0] for ent in ents if ent[1]=="DATE"])
act = " ".join([ent[0] for ent in ents_act if ent[1]=="ACTIVITY"])

if not all([loc,date]):
    pVect = returnProbabilitieVector(loc_dat, model)
    if not loc:
        loc = returnMostLikely(pVect, "LOCATION")
    if not date:
        date = returnMostLikely(pVect, "DATE")
        
# if not act and model != "english.muc.7class.distsim.crf.ser": finish when got data for ner model
#     pVect = returnProbabilitieVector(activity, model)
#     act = returnMostLikely(vect, "ACTIVITY")


Where and when are u planing to go?
To Lyon on 21th of July
And what are going to do there?
I'm going to meet friends and have a tasty burger with them


In [41]:
if not act:
    doc = nlp_small(activity)
    acts = {}
    for possible_act in doc:
        if possible_act.pos == VERB:
            children = [child.text for child in possible_act.children if not child.is_stop and child.pos_ == "NOUN"]
            if children:
                if possible_act.text not in acts.keys():
                    acts.update({possible_act.text: children})
                else:
                    acts[possible_act.text].append(children)

    
print(f"Read location is {loc}, date {date}, action - {acts}")

Read location is Lyon, date July, action - {'meet': ['friends'], 'have': ['burger']}


In [52]:
tag = {}
for pos_act in acts.keys():
    resulted_tag = getMostSimilarTag([" ".join(acts[pos_act])])
    most_likely_tag = max(resulted_tag.items(), key=operator.itemgetter(1))
    tag.update({most_likely_tag[0]: most_likely_tag[1]})

print(tag)

No embedding in wiki model
No embedding in wiki model
{'gift': 0.4827305, 'meat': 0.679823}


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
